# PyTorch on TPU

In [1]:
# # We will train the model on a GPU

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# # on silicon mac
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# model.to(device)
# for epoch in range(n_epochs):
#     for data in trainloader:
#         input, labels = data
#         inpit = input.to(device)
#         labels = labels.to(device)
#         optmizer.zero_grad()
        
#         output = model(input)
#         loss = criterion(input, labels)
#         loss.backward()
#         optimizer.step()

cuda


# PyTorch on Multiple GPUs

## Data Parallel Processing

### The multithreaded approach using `nn.DataParallel`

In [3]:
# First, we check to make we have multiple GPUs

# import torch
# import torch.nn as nn

# # if torch.cuda.device_count()      # nvidia
# if torch.backends.mps.is_available():
#     print('This machine has', torch.backends.mps.is_built(), "GPU available.")

# if torch.cuda.is_available():
#     print('This machine has', torch.cuda.device_count()), "GPU available.")
    
#     model = nn.DataParallel(model)

# model.to('mps')


# Model Optimization

In [4]:
# Let's redefine our model so that we can configure the number of the number of nodes in the fully connected layers.

import torch.nn as nn
import torch.nn. functional as F 

class Net(nn.Module):
    def __init__(self, nodes_1=120, nodes_2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # configure nodes in `fc1`
        self.fc1 = nn.Linear(16 * 5 * 5, nodes_1)
        # configure nodes in `fc2`
        self.fc2 = nn.Linear(nodes_1, nodes_2)
        self.fc3 = nn.Linear(nodes_2, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# we import the ray package and define the hyperparameter configuration
from ray import tune
import numpy as np

config = {
    "nodes_1": tune.sample_from(
        lambda _: 2 ** np.random.randint(2, 9)
    ),
    "nodes_2": tune.sample_from(
        lambda _: 2 ** np.random.randint(2, 9)
    ),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

2023-10-22 00:42:02,055	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [9]:
# load CIFAR-10 datas

import torch
import torchvision
from torchvision import transforms

def load_data(data_dir='./data/CIFAR10/'):
    train_transforms = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465),
            (0.2023, 0.1994, 0.2010)
        )
    ])
    
    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465),
            (0.2023, 0.1994, 0.2010)
        )
    ])
    
    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True,
        download=True,
        transform=train_transforms
    )
    
    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False,
        download=True,
        transform=test_transforms
    )
    
    return trainset, testset

In [10]:
# we can wrap our training loop into a function, `train_model()`

from torch import optim
from torch import nn 
from torch.utils.data import random_split

def train_model(config):
    
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    
    # make the model layer configurable
    model = Net(config['nodes_1'], config['nodes_2']).to(device=device)
    
    criterion = nn.CrossEntropyLoss()
    # make the learning rate configurable
    optimizer = optim.SGD(model.parameters(),
                          lr=config['lr'],
                          momentum=0.9)
    
    trainset, testset = load_data()
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])
    
    # make the batch size configurable
    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size = int(config['batch_size']),
        shuffle=True
    )
    
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size = int(config['batch_size']),
        shuffle=True
    )
    
    for epoch in range(10):
        
        train_loss = 0.0
        epoch_steps = 0
        for data in trainloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        val_loss = 0.0
        total = 0
        correct = 0
        for data in valloader:
            with torch.no_grad():
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                _, predicted = torch.max(
                    outputs.data, 1
                )
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
        
        print(f'epoch: {epoch} ', 
              f'train_loss : ', 
              f'{train_loss/len(trainloader)}',
              f'val_loss: ',
              f'{val_loss/len(valloader)}',
              f'val_acc: {correct/total}')
    
        tune.report(loss=(val_loss / len(valloader)), accuracy=correct/total)

In [11]:
# the scheduler determines how Ray Tune searches and selects the hyperparameters, while the reporter specifies how we’d like to view the results.
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

scheduler = ASHAScheduler(
    metric='loss',
    mode='min',
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    metric_columns=['loss', 'accuracy', 'training_iteration']
)

In [12]:
# we can run Ray Tune using the run() method.
from functools import partial

result = tune.run(
    partial(train_model),
    resources_per_trial={'cpu': 2, 'gpu': 1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter
)

2023-10-22 00:42:48,064	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-22 00:42:53,727	INFO tune.py:228 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-10-22 00:42:53,735	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-10-22 00:42:53,745	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2023-10-22 00:42:53,745	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2023-10-22 00:42:53 (running for 00:00:00.17)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/20 CPUs, 1.0/1 GPUs
Result logdir: C:/Users/woojin/ray_results/train_model_2023-10-22_00-42-53
Number of trials: 10/10 (10 PENDING)
+-------------------------+----------+-------+--------------+-------------+
| Trial name              | status   | loc   |   batch_size |          lr |
|-------------------------+----------+-------+--------------+-------------|
| train_model_7fa2d_00000 | PENDING  |       |            8 | 0.0576609   |
| train_model_7fa2d_00001 | PENDING  |       |            8 | 0.0142931   |
| train_model_7fa2d_00002 | PENDING  |       |            2 | 0.059131    |
| train_model_7fa2d_00003 | PENDING  |       |            8 | 0.000869685 |
| train_model_7fa2d_00004 | PENDING  |       |           16 | 0.000118007 |
| train_model_7fa2d_00005 | PENDING  |  

0.0%(func pid=19912) 
0.0%(func pid=19912) 
0.1%(func pid=19912) 
0.1%(func pid=19912) 
0.2%(func pid=19912) 
0.2%(func pid=19912) 
0.3%(func pid=19912) 
0.3%(func pid=19912) 
0.4%(func pid=19912) 


== Status ==
Current time: 2023-10-22 00:43:04 (running for 00:00:10.33)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/20 CPUs, 1.0/1 GPUs
Result logdir: C:/Users/woojin/ray_results/train_model_2023-10-22_00-42-53
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+
| Trial name              | status   | loc             |   batch_size |          lr |
|-------------------------+----------+-----------------+--------------+-------------|
| train_model_7fa2d_00000 | RUNNING  | 127.0.0.1:19912 |            8 | 0.0576609   |
| train_model_7fa2d_00001 | PENDING  |                 |            8 | 0.0142931   |
| train_model_7fa2d_00002 | PENDING  |                 |            2 | 0.059131    |
| train_model_7fa2d_00003 | PENDING  |                 |            8 | 0.000869685 |
| train_model_7fa2d_00004 | PENDING 

0.4%(func pid=19912) 
0.5%(func pid=19912) 
0.5%(func pid=19912) 
0.5%(func pid=19912) 
0.5%(func pid=19912) 
0.6%(func pid=19912) 
0.6%(func pid=19912) 
0.6%(func pid=19912) 
0.6%(func pid=19912) 
0.7%(func pid=19912) 
0.7%(func pid=19912) 
0.7%(func pid=19912) 
0.7%(func pid=19912) 
0.8%(func pid=19912) 
0.8%(func pid=19912) 
0.9%(func pid=19912) 
0.9%(func pid=19912) 
0.9%(func pid=19912) 
1.0%(func pid=19912) 
1.0%(func pid=19912) 
1.0%(func pid=19912) 
1.0%(func pid=19912) 
1.1%(func pid=19912) 
1.1%(func pid=19912) 


...

51.1%(func pid=5100) 
51.1%(func pid=5100) 
51.2%(func pid=5100) 
51.2%(func pid=5100) 
51.2%(func pid=5100) 
51.3%(func pid=5100) 
51.3%(func pid=5100) 
51.4%(func pid=5100) 
51.4%(func pid=5100) 
51.4%(func pid=5100) 
51.4%(func pid=5100) 
51.5%(func pid=5100) 
51.7%(func pid=5100) 
51.7%(func pid=5100) 
51.7%(func pid=5100) 
51.8%(func pid=5100) 
51.9%(func pid=5100) 
51.9%(func pid=5100) 
51.9%(func pid=5100) 
52.0%(func pid=5100) 
52.0%(func pid=5100) 
52.0%(func pid=5100) 
52.0%(func pid=5100) 
52.0%(func pid=5100) 
52.1%(func pid=5100) 


== Status ==
Current time: 2023-10-22 01:08:03 (running for 00:25:09.36)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/20 CPUs, 1.0/1 GPUs
Result logdir: C:/Users/woojin/ray_results/train_model_2023-10-22_00-42-53
Number of trials: 10/10 (2 ERROR, 7 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+
| Trial name              | status   | loc             |   batch_size |          lr |
|-------------------------+----------+-----------------+--------------+-------------|
| train_model_7fa2d_00002 | RUNNING  | 127.0.0.1:5100  |            2 | 0.059131    |
| train_model_7fa2d_00003 | PENDING  |                 |            8 | 0.000869685 |
| train_model_7fa2d_00004 | PENDING  |                 |           16 | 0.000118007 |
| train_model_7fa2d_00005 | PENDING  |                 |           16 | 0.0730253   |
| train_model_7fa2d_00006 |

52.1%(func pid=5100) 
52.1%(func pid=5100) 
52.2%(func pid=5100) 
52.2%(func pid=5100) 
52.2%(func pid=5100) 
52.2%(func pid=5100) 
52.3%(func pid=5100) 
52.3%(func pid=5100) 
52.3%(func pid=5100) 
52.3%(func pid=5100) 
52.3%(func pid=5100) 
52.4%(func pid=5100) 
52.4%(func pid=5100) 
52.4%(func pid=5100) 
52.4%(func pid=5100) 
52.4%(func pid=5100) 
52.4%(func pid=5100) 
52.5%(func pid=5100) 
52.5%(func pid=5100) 
52.5%(func pid=5100) 
52.6%(func pid=5100) 
52.6%(func pid=5100) 
52.6%(func pid=5100) 
52.7%(func pid=5100) 
52.7%(func pid=5100) 
52.7%(func pid=5100) 
52.7%(func pid=5100) 
52.8%(func pid=5100) 
52.8%(func pid=5100) 
2023-10-22 01:08:07,446	WARNING tune.py:194 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
52.9%(func pid=5100) 
52.9%(func pid=5100) 
52.9%(func pid=5100) 


== Status ==
Current time: 2023-10-22 01:08:07 (running for 00:25:13.71)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/20 CPUs, 1.0/1 GPUs
Result logdir: C:/Users/woojin/ray_results/train_model_2023-10-22_00-42-53
Number of trials: 10/10 (2 ERROR, 7 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+
| Trial name              | status   | loc             |   batch_size |          lr |
|-------------------------+----------+-----------------+--------------+-------------|
| train_model_7fa2d_00002 | RUNNING  | 127.0.0.1:5100  |            2 | 0.059131    |
| train_model_7fa2d_00003 | PENDING  |                 |            8 | 0.000869685 |
| train_model_7fa2d_00004 | PENDING  |                 |           16 | 0.000118007 |
| train_model_7fa2d_00005 | PENDING  |                 |           16 | 0.0730253   |
| train_model_7fa2d_00006 |

52.9%(func pid=5100) 
52.9%(func pid=5100) 
53.0%(func pid=5100) 
53.0%(func pid=5100) 
53.0%(func pid=5100) 
53.1%(func pid=5100) 
53.1%(func pid=5100) 
53.1%(func pid=5100) 
53.2%(func pid=5100) 
53.3%(func pid=5100) 
53.3%(func pid=5100) 
53.3%(func pid=5100) 
53.3%(func pid=5100) 
53.4%(func pid=5100) 
53.4%(func pid=5100) 
53.4%(func pid=5100) 
53.4%(func pid=5100) 
53.4%(func pid=5100) 
53.5%(func pid=5100) 
53.5%(func pid=5100) 
53.6%(func pid=5100) 
53.6%(func pid=5100) 
53.6%(func pid=5100) 
53.7%(func pid=5100) 
53.7%(func pid=5100) 
53.7%(func pid=5100) 
53.7%(func pid=5100) 
53.8%(func pid=5100) 
53.8%(func pid=5100) 
53.8%(func pid=5100) 
53.8%(func pid=5100) 
53.9%(func pid=5100) 
53.9%(func pid=5100) 
53.9%(func pid=5100) 
54.0%(func pid=5100) 
54.0%(func pid=5100) 
54.0%(func pid=5100) 
54.0%(func pid=5100) 
54.1%(func pid=5100) 
54.1%(func pid=5100) 
54.1%(func pid=5100) 
54.1%(func pid=5100) 
54.2%(func pid=5100) 
54.2%(func pid=5100) 
54.2%(func pid=5100) 
54.3%(func

In [1]:
# we can print out the hyperparameter settings that yielded the best results
best_trial = result.get_best_trial(
    'loss', 'min', 'last'
)
print('Best trial config: {}'.format(best_trial.config))
print('Best trial final validation loss: ', "{}".format(best_trial.last_result["loss"]))
print('Best trial final validation accuracy: ', "{}".format(best_trial.last_result["accuracy"]))

NameError: name 'result' is not defined

## Quantization

In [8]:
# A quick quantization method is to reduce all computation precision by half.
# Let's consider our LeNet5 model example.

import torch
from torch import nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(
            F.relu(self.conv1(x), (2, 2))
        )
        x = F.max_pool2d(
            F.relu(self.conv2(x)), 2
        )
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = LeNet5()

In [9]:
# we can inspect the data types of our model's parameters using the following code.
for n, p in model.named_parameters():
    print(n, ": ", p.dtype)

conv1.weight :  torch.float32
conv1.bias :  torch.float32
conv2.weight :  torch.float32
conv2.bias :  torch.float32
fc1.weight :  torch.float32
fc1.bias :  torch.float32
fc2.weight :  torch.float32
fc2.bias :  torch.float32
fc3.weight :  torch.float32
fc3.bias :  torch.float32


In [10]:
model = model.half()

for n, p in model.named_parameters():
    print(n, ": ", p.dtype)

conv1.weight :  torch.float16
conv1.bias :  torch.float16
conv2.weight :  torch.float16
conv2.bias :  torch.float16
fc1.weight :  torch.float16
fc1.bias :  torch.float16
fc2.weight :  torch.float16
fc2.bias :  torch.float16
fc3.weight :  torch.float16
fc3.bias :  torch.float16


# Pruning

## Pruning model example

In [12]:
# Let's consider our exmaple LeNet5 model
from torch import nn
import torch.nn.functional as F

class LeNet5(nn.Module):
		def __init__(self):
				super(LeNet5, self).__init__()
				self.conv1 = nn.Conv2d(3, 6, 5)
				self.conv2 = nn.Conv2d(6, 16, 5)
				self.fc1 = nn.Linear(16 * 5 * 5, 120)
				self.fc2 = nn.Linear(120, 84)
				self.fc3 = nn.Linear(84, 10)

		def forward(self, x):
				x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
				x = F.max_pool2d(F.relu(self.conv2(x)), 2)
				x = x.view(-1, int(x.nelement() / x.shape[0]))
				x = F.relu(self.fc1(x))
				x = F.relu(self.fc2(x))
				x = self.fc3(x)
				return x

In [13]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = LeNet5().to(device)

print(list(model.conv1.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 1.1536e-01,  8.1601e-03,  7.1029e-02,  1.0038e-01,  5.0458e-02],
          [-4.6634e-02,  6.6983e-02, -9.7700e-02,  5.6592e-02, -7.3028e-02],
          [-5.3506e-02,  2.9965e-02, -3.8252e-02,  4.4904e-02,  9.2438e-02],
          [ 5.8594e-02, -7.1564e-02,  1.0680e-01,  4.4284e-04, -1.1001e-01],
          [ 2.8073e-02,  4.9907e-02, -1.5619e-02, -7.3089e-03, -2.0224e-03]],

         [[ 1.0776e-01,  5.3707e-02,  1.0536e-01,  1.1449e-01, -8.0869e-03],
          [ 9.7646e-02,  5.9824e-02,  1.0735e-01, -6.2516e-02,  5.7925e-02],
          [-7.9726e-02, -7.2015e-02,  3.8788e-02,  6.9317e-02, -1.0046e-01],
          [-4.3681e-02, -3.6045e-02,  5.1272e-02,  8.5856e-02,  1.1424e-01],
          [ 6.1476e-02,  6.9969e-02, -8.9264e-02,  7.6909e-02, -1.0468e-01]],

         [[-6.8655e-02,  2.2079e-02,  9.0891e-02, -8.2564e-02, -4.2875e-03],
          [ 6.6851e-02, -8.7167e-02,  9.7841e-02,  4.5263e-02, -8.8098e-02],
          [ 1.3923e-02,  2.4221e-02,  